# Project 3: Smart Onboarding

Build an automated onboarding check-in system. An employee fills out a survey, an AI agent classifies the response, and the workflow routes it to the right follow-up — all without a human reading every form.

The main lesson: **a Switch node can route by dropdown values, but only AI can read free text and detect problems the employee didn't explicitly flag.**

| What you'll learn | Where it comes from |
|-------------------|--------------------|  
| **Form Trigger** — collect structured input via a web form | New node (not used in previous chapters) |
| **AI classification with override** — the agent can contradict the dropdown | Builds on the Routing chapter (Switch + LLM) |
| **Referencing upstream nodes** — access form data after the Switch | Builds on Core Concepts (expressions) |

**Workflow in this chapter:**

| File | What it does | GitHub Link |
|------|-------------|-------------|
| `12_onboarding_checkin.json` | Form survey → AI classification → routed email output | [View](https://github.com/ezponda/ai-agents-course/blob/main/courses/n8n_no_code/book/_static/workflows/12_onboarding_checkin.json) |

**Credentials needed:** OpenRouter API key (Settings → Credentials).

---

## Why AI? The Free-Text Problem

A survey with only dropdown questions is easy to route — a Switch node matches the value and sends data down the right path. No AI needed.

But most real surveys include a free-text field: *"Tell us more."* And that's where the problems hide:

| Dropdown answer | Free-text comment | What should happen |
|----------------|-------------------|-------------------|
| Great | *(empty)* | Send encouraging email |
| Good | "Everything is fine, my team is helpful" | Send encouraging email |
| **Good** | **"Everything is fine, but a colleague keeps making comments that make me uncomfortable"** | **Alert the manager immediately** |
| Bad | "I don't have access to email or Slack yet" | Send help + alert manager |

Row 3 is the key case. The dropdown says "Good", but the comment reveals a serious concern. A Switch node sees "Good" and routes to the happy path. An AI agent reads the text and overrides the classification.

This is the difference between automation and intelligence.

---

## The Workflow

```
┌──────────────┐    ┌──────────────┐    ┌───────────────┐    ┌─────────────────┐    ┌─────────────────┐
│  Onboarding  │───▶│  Classify    │───▶│    Clean      │───▶│   Route by      │──▶│ Output —        │
│  Check-in    │    │  Response    │    │ Classification│    │ Classification  │   │ On Track        │
│  (Form)      │    │  (Agent)     │    │    (Set)      │    │   (Switch)      │──▶│ Output —        │
└──────────────┘    └──────────────┘    └───────────────┘    │                 │   │ Needs Help      │
                          ┊                                   │                 │──▶│ Output —        │
                     Chat Model                               └─────────────────┘   │ At Risk         │
                                                                                    └─────────────────┘
```

- **Form Trigger** — a web form the employee fills out (name, department, rating, free text)
- **AI Agent** — reads the rating + free text, outputs a single classification word
- **Clean Classification** — trims whitespace and lowercases the agent's output
- **Switch** — routes to one of three branches: `on_track`, `needs_help`, `at_risk`
- **Output nodes** — prepare the email content for each path (employee email + manager alert for `at_risk`)

**File:** [`12_onboarding_checkin.json`](https://github.com/ezponda/ai-agents-course/blob/main/courses/n8n_no_code/book/_static/workflows/12_onboarding_checkin.json)

> **Import via URL** (copy and paste in n8n → Import from URL):
> ```
> https://raw.githubusercontent.com/ezponda/ai-agents-course/main/courses/n8n_no_code/book/_static/workflows/12_onboarding_checkin.json
> ```
>
> **Download:** {download}`12_onboarding_checkin.json <_static/workflows/12_onboarding_checkin.json>`

::::{dropdown} 🛠️ Build this workflow from scratch (step-by-step)
:color: secondary

### Step 1: Create a new workflow

1. Click **Workflows** → **Add Workflow**
2. Rename it to "Onboarding Check-in"

### Step 2: Add the Form Trigger

1. Delete the default Manual Trigger
2. Add **Form Trigger** (search for "On form submission")
3. Set **Form Title** to `Onboarding Check-in`
4. Add these fields:

| Field Label | Type | Required | Options (for dropdowns) |
|------------|------|----------|------------------------|
| `Full name` | Text | Yes | — |
| `Email` | Text | Yes | — |
| `Department` | Dropdown | Yes | `Engineering, Marketing, Sales, Support, Operations` |
| `Onboarding week` | Dropdown | Yes | `Week 1, Week 2, Week 3, Week 4+` |
| `How is your onboarding going?` | Dropdown | Yes | `Great, Good, Okay, Bad` |
| `Tell us more (optional)` | Textarea | No | — |
| `Manager email` | Text | Yes | — |

### Step 3: Add the AI Agent

1. Add **AI Agent** → rename to `Classify Response`
2. Configure:
   - **Source for Prompt**: `Define below`
   - **Prompt** (Expression):
     ```
     Rating: {{ $json['How is your onboarding going?'] }}
     Comment: {{ $json['Tell us more (optional)'] }}
     ```
   - Open **Options** → add **System Message** (paste the full system prompt from the "System Prompt" section below)

### Step 4: Add the Chat Model (sub-node)

1. Click **+ Chat Model** at the bottom of the AI Agent node
2. Select **OpenRouter Chat Model**
3. Choose your credential
4. Model: `openai/gpt-4o-mini`

### Step 5: Add Clean Classification

1. Add **Edit Fields** → rename to `Clean Classification`
2. Add one field:

| Name | Value (Expression) |
|------|-------------------|
| `classification` | `{{ $json.output.trim().toLowerCase() }}` |

### Step 6: Add the Switch

1. Add **Switch** → rename to `Route by Classification`
2. Add three rules, each matching `{{ $json.classification }}`:

| Rule | Condition | Output name |
|------|-----------|-------------|
| 1 | Equals `on_track` | `on_track` |
| 2 | Equals `needs_help` | `needs_help` |
| 3 | Equals `at_risk` | `at_risk` |

3. Set **Fallback Output** to `Extra Output` (catches unexpected values)

### Step 7: Add Output nodes

Add three **Edit Fields** nodes, one per Switch output:

**Output — On Track** (connected to the `on_track` output):

| Name | Value |
|------|-------|
| `to` | `{{ $('Onboarding Check-in').item.json['Email'] }}` |
| `subject` | `Your onboarding check-in — looking great!` |
| `body` | `Hi {{ $('Onboarding Check-in').item.json['Full name'] }}, Great to hear your onboarding is going well! Keep up the great work. Best, The People Team` |
| `classification` | `on_track` |

**Output — Needs Help** (connected to the `needs_help` output):

| Name | Value |
|------|-------|
| `to` | `{{ $('Onboarding Check-in').item.json['Email'] }}` |
| `subject` | `Your onboarding check-in — we want to help` |
| `body` | `Hi {{ $('Onboarding Check-in').item.json['Full name'] }}, Thanks for your honest feedback. We want to make sure you have everything you need. Someone from our team will reach out shortly. Best, The People Team` |
| `classification` | `needs_help` |

**Output — At Risk** (connected to the `at_risk` output):

| Name | Value |
|------|-------|
| `to` | `{{ $('Onboarding Check-in').item.json['Email'] }}` |
| `subject` | `Your onboarding check-in — we're here for you` |
| `body` | `Hi {{ $('Onboarding Check-in').item.json['Full name'] }}, Thank you for sharing. Your feedback matters, and your manager has been notified. Someone will reach out within 24 hours. Best, The People Team` |
| `manager_to` | `{{ $('Onboarding Check-in').item.json['Manager email'] }}` |
| `manager_subject` | `Onboarding alert: {{ $('Onboarding Check-in').item.json['Full name'] }}` |
| `manager_body` | `{{ $('Onboarding Check-in').item.json['Full name'] }} ({{ $('Onboarding Check-in').item.json['Department'] }}, {{ $('Onboarding Check-in').item.json['Onboarding week'] }}) was flagged as at_risk. Rating: {{ $('Onboarding Check-in').item.json['How is your onboarding going?'] }}. Comment: {{ $('Onboarding Check-in').item.json['Tell us more (optional)'] }}. Please follow up within 24 hours.` |
| `classification` | `at_risk` |

### Step 8: Connect the nodes

```
Onboarding Check-in → Classify Response → Clean Classification → Route by Classification
                                                                   ├─ on_track → Output — On Track
                                                                   ├─ needs_help → Output — Needs Help
                                                                   └─ at_risk → Output — At Risk
```

### Step 9: Test

1. Click **Test workflow** — a form opens in a new tab
2. Fill it in with: Full name = `Test User`, Rating = `Good`, Tell us more = `My laptop hasn't arrived yet and I can't access any tools`
3. Check which Output node received the data — it should be **Output — Needs Help** or **Output — At Risk** (the agent overrides "Good" because the comment describes a real problem)

::::

### Node-by-Node Walkthrough

| Node | Type | What it does |
|------|------|-------------|
| **Onboarding Check-in** | Form Trigger | Collects employee survey responses via a web form |
| **Classify Response** | AI Agent | Reads the rating + free text, outputs one word: `on_track`, `needs_help`, or `at_risk` |
| **Clean Classification** | Set | Trims whitespace and lowercases the classification |
| **Route by Classification** | Switch | Sends data to the matching branch |
| **Output — On Track** | Set | Prepares encouraging email content |
| **Output — Needs Help** | Set | Prepares helpful email content |
| **Output — At Risk** | Set | Prepares supportive email + manager alert |

**Sub-node:**

| Sub-node | Type | Purpose |
|----------|------|--------|
| **OpenRouter Chat Model** | Chat Model | Provides the LLM for classification |

### Data Flow

```
INPUT                          AGENT                               SWITCH + OUTPUT
─────                          ─────                               ───────────────
Form: { "Full name": "María García",
        "Email": "maria@company.com",
        "How is your onboarding going?": "Good",
        "Tell us more (optional)": "A colleague keeps making comments that make me uncomfortable",
        "Manager email": "boss@company.com" }
    ↓
Agent input: "Rating: Good\nComment: A colleague keeps making comments that make me uncomfortable"
Agent output: { output: "at_risk" }   ← override! Dropdown said "Good" but text reveals a problem
    ↓
Clean: { classification: "at_risk" }
    ↓
Switch → at_risk branch
    ↓
Output — At Risk: { to: "maria@company.com",
                    subject: "Your onboarding check-in — we're here for you",
                    body: "Hi María García, ...",
                    manager_to: "boss@company.com",
                    manager_subject: "Onboarding alert: María García",
                    manager_body: "María García (Engineering, Week 2) was flagged as at_risk..." }
```

::::{dropdown} 🔍 See detailed data transformation at each step
:color: info

```
┌────────────────┐     ┌────────────────┐     ┌────────────────┐     ┌────────────────┐     ┌────────────────┐
│ Onboarding     │────▶│ Classify       │────▶│ Clean          │────▶│ Route by       │────▶│ Output —       │
│ Check-in       │     │ Response       │     │ Classification │     │ Classification │     │ At Risk        │
└────────────────┘     └────────────────┘     └────────────────┘     └────────────────┘     └────────────────┘
        │                      │                      │                      │                      │
        ▼                      ▼                      ▼                      ▼                      ▼
┌────────────────┐     ┌────────────────┐     ┌────────────────┐     Routes to the       ┌────────────────┐
│ Full name:     │     │ output:        │     │ classification:│     "at_risk" branch     │ to: maria@...  │
│ "María García" │     │ "at_risk"      │     │ "at_risk"      │                          │ subject: ...   │
│ Email: ...     │     │                │     │                │                          │ body: ...      │
│ Rating: "Good" │     │ (overrode the  │     │ (trimmed +     │                          │ manager_to:... │
│ Comment: "..." │     │  dropdown!)    │     │  lowercased)   │                          │ manager_body:..│
└────────────────┘     └────────────────┘     └────────────────┘                          └────────────────┘
```

**Key insight:** The form data is still accessible from any downstream node using `$('Onboarding Check-in').item.json['field']`. The Switch only uses the `classification` field for routing — it doesn't erase the original data.

::::

---

## The System Prompt

The agent's job is simple: read a rating and a comment, output one word. The system prompt defines the classification rules and — crucially — when to override the dropdown.

### Classify Response — System Message

```
You classify employee onboarding check-in responses.

Input: a rating (Great / Good / Okay / Bad) and an optional free-text comment.

Rules:
1. If there is no comment, classify based on the rating:
   - Great or Good → on_track
   - Okay → needs_help
   - Bad → at_risk

2. If there IS a comment, read it carefully. Override the rating if:
   - The comment describes real problems despite a positive rating → needs_help or at_risk
   - The comment mentions harassment, safety, or serious discomfort → always at_risk

Output ONLY one word: on_track, needs_help, or at_risk
No explanation. No other text. Just the classification word.
```

### Classify Response — Prompt (Expression)

```
Rating: {{ $json['How is your onboarding going?'] }}
Comment: {{ $json['Tell us more (optional)'] }}
```

### Why this prompt works

| Element | Why it matters |
|---------|---------------|
| Explicit rating values (Great / Good / Okay / Bad) | The agent knows exactly what to expect |
| Default mapping (Great → on_track, etc.) | Handles the common case (empty comment) |
| Override rules | The core value — read the text, contradict the dropdown if needed |
| "Output ONLY one word" | The Switch needs a clean string to match — no extra text |

### What if the agent outputs something unexpected?

The **Clean Classification** node handles minor variations: `" on_track "` becomes `"on_track"`, `"On_Track"` becomes `"on_track"`. If the agent still outputs something the Switch can't match, the **fallback output** catches it — same path as `at_risk` (safe default).

---

## Test Scenarios

Run the workflow with these three cases to see the classification in action.

### Case 1: No override needed

| Field | Value |
|-------|-------|
| Full name | `Alex Chen` |
| Email | `alex@company.com` |
| Department | `Engineering` |
| Onboarding week | `Week 1` |
| Rating | `Bad` |
| Comment | `I don't have access to email or Slack yet. Nobody has shown me where anything is.` |
| Manager email | `manager@company.com` |

**Expected:** `at_risk` — no override needed, the dropdown already says Bad.

### Case 2: Override — the key demo

| Field | Value |
|-------|-------|
| Full name | `María García` |
| Email | `maria@company.com` |
| Department | `Marketing` |
| Onboarding week | `Week 2` |
| Rating | `Good` |
| Comment | `Everything is fine, but a colleague keeps making comments that make me uncomfortable.` |
| Manager email | `boss@company.com` |

**Expected:** `at_risk` — the agent overrides "Good" because the comment reveals a harassment concern. The Output — At Risk node prepares a manager alert.

### Case 3: No comment, trust the dropdown

| Field | Value |
|-------|-------|
| Full name | `Sam Johnson` |
| Email | `sam@company.com` |
| Department | `Sales` |
| Onboarding week | `Week 3` |
| Rating | `Great` |
| Comment | *(leave empty)* |
| Manager email | `manager@company.com` |

**Expected:** `on_track` — no comment, so the agent trusts the dropdown.

---

## What to Observe + Key Takeaways

After running the three test cases:

**1. Case 2 is why the agent exists**

Without AI, a Switch on the dropdown sends "Good" to the happy path. The harassment concern goes unnoticed. The agent reads the free text and overrides. This is the difference between routing and classification.

**2. The agent outputs one word**

The system prompt says "Output ONLY one word". This makes the Switch reliable — it matches an exact string. If the agent wrote a paragraph, the Switch couldn't route it.

**3. Clean Classification is a safety net**

LLMs sometimes add whitespace or change capitalization. `$json.output.trim().toLowerCase()` handles this. The fallback output on the Switch catches anything else.

**4. Form data is accessible everywhere**

Even after the Switch, you can reference `$('Onboarding Check-in').item.json['Full name']`. This is how the Output nodes build personalized emails without passing every field through every node.

**5. The Output nodes are templates, not sent emails**

The workflow prepares the email content (to, subject, body) but doesn't send it. This lets you inspect the output before connecting a real email service. See Challenge 1.

---

## Try It Yourself

### Challenge 1: Send Real Emails with Gmail

Replace each Output Edit Fields node with a **Gmail** node (or **Send Email** node). Use your Google account credentials.

For the `at_risk` branch, you need two Gmail nodes in sequence: one to the employee, one to the manager.

**Hint:** The Gmail node has `To`, `Subject`, and `Message` fields. Use the same expressions from the Output nodes.

**Done when:** you receive a real email after submitting the form.

### Challenge 2: Log to Google Sheets

Add a **Google Sheets** node at the end of each branch. Append a row with: timestamp, name, department, week, rating, classification.

All three branches connect to the same Google Sheets node. After a few test runs, open the Sheet and see the log.

**Hint:** Use `{{ $now.toFormat('yyyy-MM-dd HH:mm') }}` for the timestamp. Follow the same pattern from the tips video (Prompt Chaining chapter).

**Done when:** the Google Sheet has rows from all three classification types.

### Challenge 3: Add Follow-up with Wait

After the email in each branch, add a **Wait** node, then a second email reminding the employee to fill in their next check-in. Use different wait times:

| Branch | Wait time (production) | Wait time (testing) |
|--------|----------------------|--------------------|
| on_track | 7 days | 1 minute |
| needs_help | 3 days | 30 seconds |
| at_risk | 1 day | 15 seconds |

The follow-up email should include the Form Trigger's production URL so the employee can fill in the next check-in.

**Hint:** The Wait node is covered in the Human-in-the-Loop chapter. Set it to `After time interval`.

**Done when:** you submit the form, receive the first email, wait, then receive a follow-up email with the form link.

### Challenge 4: Upgrade to Agent with History

Add a **Google Sheets Tool** to the agent so it can read previous check-ins for the same employee. Update the system prompt to check for trends:

- "If the employee has been Okay for 2+ consecutive weeks, classify as at_risk"
- "If the employee improved from Bad to Good, mention it in a positive note"

This is what makes it a true agent (not just a chain) — it decides whether to use the tool based on context.

**Done when:** an employee who submits "Okay" three weeks in a row gets classified as `at_risk` on the third submission.

## Summary

| Concept | What You Learned |
|---------|------------------|
| **Form Trigger** | Collect structured input via a web form — no external tools needed |
| **AI classification** | An agent can read free text and output a single category word |
| **Override logic** | The agent can contradict a dropdown when the free text reveals something the dropdown missed |
| **Clean Classification** | Always clean LLM output (trim, lowercase) before using it in a Switch |
| **Referencing upstream nodes** | `$('Node Name').item.json['field']` accesses data from any earlier node |
| **Template emails** | Edit Fields nodes prepare email content that can be inspected before sending |

**Key expressions:**
- `{{ $json['How is your onboarding going?'] }}` — access a form field in the next node
- `{{ $json.output.trim().toLowerCase() }}` — clean the agent's output
- `{{ $('Onboarding Check-in').item.json['Full name'] }}` — reference form data from any downstream node
- `{{ $now.toFormat('yyyy-MM-dd HH:mm') }}` — current timestamp (for Google Sheets logging)

**Docs:**
- [Form Trigger](https://docs.n8n.io/integrations/builtin/core-nodes/n8n-nodes-base.formtrigger/)
- [Switch node](https://docs.n8n.io/integrations/builtin/core-nodes/n8n-nodes-base.switch/)
- [Gmail node](https://docs.n8n.io/integrations/builtin/app-nodes/n8n-nodes-base.gmail/)
- [Wait node](https://docs.n8n.io/integrations/builtin/core-nodes/n8n-nodes-base.wait/)